In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Installer les bibliothèques nécessaires
!pip install numpy pandas opencv-python scikit-learn matplotlib seaborn gdown

# Télécharger le dataset depuis Google Drive
!gdown --id 1g03sIzi8F855KRMi0wmJesdiVQe219nY

# Dézipper le fichier
import zipfile
with zipfile.ZipFile('amhcd-data-64.zip', 'r') as zip_ref:
    zip_ref.extractall('amhcd-data-64')

# Vérifier le contenu du dossier
print(os.listdir('amhcd-data-64'))

# Fonctions d’activation
def relu(x):
    """ReLU activation: max(0, x)"""
    assert isinstance(x, np.ndarray), "Input to ReLU must be a numpy array"
    result = np.maximum(0, x)
    assert np.all(result >= 0), "ReLU output must be non-negative"
    return result

def relu_derivative(x):
    """Derivative of ReLU: 1 if x > 0, else 0"""
    assert isinstance(x, np.ndarray), "Input to ReLU derivative must be a numpy array"
    result = np.where(x > 0, 1, 0)
    assert np.all((result == 0) | (result == 1)), "ReLU derivative must be 0 or 1"
    return result

def leaky_relu(x, alpha=0.01):
    """Leaky ReLU activation"""
    assert isinstance(x, np.ndarray), "Input to Leaky ReLU must be a numpy array"
    return np.where(x > 0, x, alpha * x)

def leaky_relu_derivative(x, alpha=0.01):
    """Derivative of Leaky ReLU"""
    assert isinstance(x, np.ndarray), "Input to Leaky ReLU derivative must be a numpy array"
    return np.where(x > 0, 1, alpha)

def softmax(x):
    """Softmax activation: exp(x) / sum(exp(x))"""
    assert isinstance(x, np.ndarray), "Input to softmax must be a numpy array"
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Éviter le débordement numérique
    result = exp_x / np.sum(exp_x, axis=1, keepdims=True)
    assert np.all((result >= 0) & (result <= 1)), "Softmax output must be in [0, 1]"
    assert np.allclose(np.sum(result, axis=1), 1), "Softmax output must sum to 1 per sample"
    return result

# Classe MultiClassNeuralNetwork
class MultiClassNeuralNetwork:
    def _init_(self, layer_sizes, learning_rate=0.001):
        """Initialize the neural network with given layer sizes and learning rate."""
        assert isinstance(layer_sizes, list) and len(layer_sizes) >= 2, "layer_sizes must be a list with at least 2 elements"
        assert all(isinstance(size, int) and size > 0 for size in layer_sizes), "All layer sizes must be positive integers"
        assert isinstance(learning_rate, (int, float)) and learning_rate > 0, "Learning rate must be a positive number"

        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.weights = []
        self.biases = []
        self.m_w = []  # Momentum pour Adam
        self.v_w = []  # RMSProp pour Adam
        self.t = 0     # Compteur d'itérations pour Adam

        np.random.seed(42)
        for i in range(len(layer_sizes) - 1):
            w = np.random.randn(layer_sizes[i], layer_sizes[i + 1]) * 0.01
            b = np.zeros((1, layer_sizes[i + 1]))
            self.weights.append(w)
            self.biases.append(b)
            self.m_w.append(np.zeros_like(w))
            self.v_w.append(np.zeros_like(w))

    def forward(self, X):
        """Forward propagation."""
        assert isinstance(X, np.ndarray), "Input X must be a numpy array"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"

        self.activations = [X]
        self.z_values = []

        for i in range(len(self.weights) - 1):
            z = np.dot(self.activations[i], self.weights[i]) + self.biases[i]
            self.z_values.append(z)
            self.activations.append(leaky_relu(z))

        z = np.dot(self.activations[-1], self.weights[-1]) + self.biases[-1]
        self.z_values.append(z)
        output = softmax(z)
        self.activations.append(output)
        return self.activations[-1]

    def compute_loss(self, y_true, y_pred):
        """Categorical Cross-Entropy."""
        assert isinstance(y_true, np.ndarray) and isinstance(y_pred, np.ndarray), "Inputs to loss must be numpy arrays"
        assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"

        y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
        loss = -np.mean(np.sum(y_true * np.log(y_pred), axis=1))
        assert not np.isnan(loss), "Loss computation resulted in NaN"
        return loss

    def compute_accuracy(self, y_true, y_pred):
        """Compute accuracy."""
        assert isinstance(y_true, np.ndarray) and isinstance(y_pred, np.ndarray), "Inputs to accuracy must be numpy arrays"
        assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"

        predictions = np.argmax(y_pred, axis=1)
        true_labels = np.argmax(y_true, axis=1)
        accuracy = np.mean(predictions == true_labels)
        assert 0 <= accuracy <= 1, "Accuracy must be between 0 and 1"
        return accuracy

    def backward(self, X, y, outputs):
        """Backpropagation."""
        assert isinstance(X, np.ndarray) and isinstance(y, np.ndarray) and isinstance(outputs, np.ndarray), "Inputs to backward must be numpy arrays"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
        assert y.shape == outputs.shape, "y and outputs must have the same shape"

        m = X.shape[0]
        self.d_weights = [np.zeros_like(w) for w in self.weights]
        self.d_biases = [np.zeros_like(b) for b in self.biases]

        dZ = outputs - y  # Gradient pour softmax + cross-entropy
        self.d_weights[-1] = (self.activations[-2].T @ dZ) / m
        self.d_biases[-1] = np.sum(dZ, axis=0, keepdims=True) / m

        for i in range(len(self.weights) - 2, -1, -1):
            dZ = np.dot(dZ, self.weights[i + 1].T) * leaky_relu_derivative(self.z_values[i])
            self.d_weights[i] = (self.activations[i].T @ dZ) / m
            self.d_biases[i] = np.sum(dZ, axis=0, keepdims=True) / m

        # Régularisation L2
        for i in range(len(self.weights)):
            self.d_weights[i] += (0.01 / m) * self.weights[i]

        self.update_with_adam()

    def update_with_adam(self):
        """Update weights and biases using Adam optimizer."""
        self.t += 1
        beta1, beta2 = 0.9, 0.999
        epsilon = 1e-8
        self.m_w = [beta1 * m + (1 - beta1) * dw for m, dw in zip(self.m_w, self.d_weights)]
        self.v_w = [beta2 * v + (1 - beta2) * dw**2 for v, dw in zip(self.v_w, self.d_weights)]
        m_w_hat = [m / (1 - beta1**self.t) for m in self.m_w]
        v_w_hat = [v / (1 - beta2**self.t) for v in self.v_w]
        self.weights = [w - self.learning_rate * m / (np.sqrt(v) + epsilon) for w, m, v in zip(self.weights, m_w_hat, v_w_hat)]
        self.biases = [b - self.learning_rate * db / (np.sqrt(np.sum(db**2) / db.size + epsilon)) for b, db in zip(self.biases, self.d_biases)]

    def train(self, X, y, X_val, y_val, epochs, batch_size):
        """Train the neural network using mini-batch SGD, with validation."""
        assert isinstance(X, np.ndarray) and isinstance(y, np.ndarray), "X and y must be numpy arrays"
        assert isinstance(X_val, np.ndarray) and isinstance(y_val, np.ndarray), "X_val and y_val must be numpy arrays"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"
        assert y.shape[1] == self.layer_sizes[-1], f"Output dimension ({y.shape[1]}) must match output layer size ({self.layer_sizes[-1]})"
        assert isinstance(epochs, int) and epochs > 0, "Epochs must be a positive integer"
        assert isinstance(batch_size, int) and batch_size > 0, "Batch size must be a positive integer"

        train_losses = []
        val_losses = []
        train_accuracies = []
        val_accuracies = []

        for epoch in range(epochs):
            indices = np.random.permutation(X.shape[0])
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            epoch_loss = 0
            for i in range(0, X.shape[0], batch_size):
                X_batch = X_shuffled[i:i + batch_size]
                y_batch = y_shuffled[i:i + batch_size]
                outputs = self.forward(X_batch)
                epoch_loss += self.compute_loss(y_batch, outputs)
                self.backward(X_batch, y_batch, outputs)

            train_loss = epoch_loss / (X.shape[0] // batch_size)
            train_pred = self.forward(X)
            train_accuracy = self.compute_accuracy(y, train_pred)
            val_pred = self.forward(X_val)
            val_loss = self.compute_loss(y_val, val_pred)
            val_accuracy = self.compute_accuracy(y_val, val_pred)

            train_losses.append(train_loss)
            val_losses.append(val_loss)
            train_accuracies.append(train_accuracy)
            val_accuracies.append(val_accuracy)

            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, "
                      f"Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}")

        return train_losses, val_losses, train_accuracies, val_accuracies

    def predict(self, X):
        """Predict class labels."""
        assert isinstance(X, np.ndarray), "Input X must be a numpy array"
        assert X.shape[1] == self.layer_sizes[0], f"Input dimension ({X.shape[1]}) must match input layer size ({self.layer_sizes[0]})"

        outputs = self.forward(X)
        predictions = np.argmax(outputs, axis=1)
        assert predictions.shape == (X.shape[0],), "Predictions have incorrect shape"
        return predictions

# Fonctions de prétraitement et augmentation
def load_and_preprocess_image(image_path, target_size=(32, 32)):
    """Load and preprocess an image: convert to grayscale, resize, normalize"""
    assert os.path.exists(image_path), f"Image not found: {image_path}"
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    assert img is not None, f"Failed to load image: {image_path}"
    img = cv2.resize(img, target_size)
    img = img.astype(np.float32) / 255.0  # Normalisation
    return img

def augment_image(img, target_size=(32, 32)):
    """Augment image with random rotation and translation."""
    # Random rotation
    angle = np.random.uniform(-15, 15)
    M = cv2.getRotationMatrix2D((16, 16), angle, 1)
    img = cv2.warpAffine(img, M, target_size)
    # Random translation
    tx, ty = np.random.uniform(-3, 3, 2)
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    img = cv2.warpAffine(img, M, target_size)
    return img.flatten()

# Chargement des données
data_dir = os.path.join('amhcd-data-64/amhcd-data-64', 'tifinagh-images/')

try:
    labels_df = pd.read_csv(os.path.join(data_dir, 'labels-map.csv'))
    assert 'image_path' in labels_df.columns and 'label' in labels_df.columns, "CSV must contain 'image_path' and 'label' columns"
except FileNotFoundError:
    print("labels-map.csv not found. Please check the dataset structure.")
    image_paths = []
    labels = []
    for label_dir in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label_dir)
        if os.path.isdir(label_path):
            for img_name in os.listdir(label_path):
                image_paths.append(os.path.join(label_path, img_name))
                labels.append(label_dir)
    labels_df = pd.DataFrame({'image_path': image_paths, 'label': labels})

assert not labels_df.empty, "No data loaded. Check dataset files."
print(f"Loaded {len(labels_df)} samples with {labels_df['label'].nunique()} unique classes.")

label_encoder = LabelEncoder()
labels_df['label_encoded'] = label_encoder.fit_transform(labels_df['label'])
num_classes = len(label_encoder.classes_)

# Charger et prétraiter toutes les images
X = np.array([load_and_preprocess_image(path, (32, 32)).flatten() for path in labels_df['image_path']])
y = labels_df['label_encoded'].values

# Vérifier les dimensions
assert X.shape[0] == y.shape[0], "Mismatch between number of images and labels"
assert X.shape[1] == 32 * 32, f"Expected flattened image size of {32*32}, got {X.shape[1]}"

# Validation croisée
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
fold_accuracies = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
    print(f"Fold {fold + 1}/5")
    X_train_fold, X_val_fold = X[train_idx], X[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]
    one_hot_encoder = OneHotEncoder(sparse_output=False)
    y_train_one_hot_fold = one_hot_encoder.fit_transform(y_train_fold.reshape(-1, 1))
    y_val_one_hot_fold = one_hot_encoder.transform(y_val_fold.reshape(-1, 1))
    
    # Augmenter X_train_fold
    X_train_aug = np.array([augment_image(img.reshape(32, 32)) for img in X_train_fold])
    y_train_aug = np.tile(y_train_fold, (len(X_train_aug) // len(X_train_fold), 1))
    X_train_fold = np.vstack((X_train_fold, X_train_aug))
    y_train_one_hot_fold = np.vstack((y_train_one_hot_fold, one_hot_encoder.transform(y_train_aug.reshape(-1, 1))))
    
    # Entraîner le modèle
    layer_sizes = [X_train_fold.shape[1], 128, 64, 16, num_classes]
    learning_rates = [0.0001]
    batch_sizes = [64]
    best_accuracy = 
    
    for lr in learning_rates:
        for bs in batch_sizes:
            print(f"Training with learning_rate={lr}, batch_size={bs}")
            nn = MultiClassNeuralNetwork(layer_sizes, learning_rate=lr)
            train_losses, val_losses, train_accuracies, val_accuracies = nn.train(
                X_train_fold, y_train_one_hot_fold, X_val_fold, y_val_one_hot_fold, epochs=100, batch_size=bs)
            val_accuracy = val_accuracies[-1]
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
    
    y_pred_fold = nn.predict(X_val_fold)
    fold_accuracy = np.mean(y_pred_fold == y_val_fold)
    fold_accuracies.append(fold_accuracy)
    print(f"Fold {fold + 1} Accuracy: {fold_accuracy:.4f}")

print(f"Mean CV Accuracy: {np.mean(fold_accuracies):.4f} (±{np.std(fold_accuracies):.4f})")

# Évaluation finale sur le test set (optionnel, si nécessaire)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, stratify=y_temp, random_state=42)
one_hot_encoder = OneHotEncoder(sparse_output=False)
y_train_one_hot = one_hot_encoder.fit_transform(y_train.reshape(-1, 1))
y_val_one_hot = one_hot_encoder.transform(y_val.reshape(-1, 1))
y_test_one_hot = one_hot_encoder.transform(y_test.reshape(-1, 1))

nn = MultiClassNeuralNetwork(layer_sizes, learning_rate=0.001)
train_losses, val_losses, train_accuracies, val_accuracies = nn.train(
    X_train, y_train_one_hot, X_val, y_val_one_hot, epochs=100, batch_size=32)

y_pred = nn.predict(X_test)
print("\nRapport de classification (Test set):")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Matrice de confusion
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_encoder.classes_, yticklabels=label_encoder.classes_)
plt.title('Matrice de confusion (Test set)')
plt.xlabel('Prédiction')
plt.ylabel('Réel')
plt.savefig('confusion_matrix.png')
plt.close()

# Analyse de la matrice de confusion
print("Analyse de la matrice de confusion :")
for i in range(len(label_encoder.classes_)):
    row_sum = np.sum(cm[i, :])
    if row_sum > 0:
        accuracy_class = cm[i, i] / row_sum
        if accuracy_class < 0.8:
            print(f"Classe {label_encoder.classes_[i]} : Précision = {accuracy_class:.2f} (problématique)")

# Courbes de perte et d’accuracy
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.plot(train_losses, label='Train Loss')
ax1.plot(val_losses, label='Validation Loss')
ax1.set_title('Courbe de perte')
ax1.set_xlabel('Époque')
ax1.set_ylabel('Perte')
ax1.legend()

ax2.plot(train_accuracies, label='Train Accuracy')
ax2.plot(val_accuracies, label='Validation Accuracy')
ax2.set_title('Courbe de précision')
ax2.set_xlabel('Époque')
ax2.set_ylabel('Précision')
ax2.legend()

plt.tight_layout()
fig.savefig('loss_accuracy_plot.png')
plt.close()